In this notebook we consider the KOI list from Jason Rowe to see if we can identify for candidates that meet our criteria for cold, icy worlds

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import astropy
from astropy import constants as ac
import lightkurve as lk
from scipy.optimize import minimize
from astropy.io import fits
from astropy.table import Table
from lightkurve import search_lightcurvefile
from lightkurve import search_lightcurve
from lightkurve import search_targetpixelfile
import exoplanet as xo

In [2]:
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
rcParams["figure.dpi"] = 100
rcParams["font.size"] = 18
rcParams["figure.figsize"] = [8, 5]
rcParams["font.family"] = "sans-serif"
rcParams["font.sans-serif"] = ["Computer Modern Sans Serif"]
rcParams["text.usetex"] = True
plt.rcParams['axes.linewidth']=1.5

In [3]:
jr = pd.read_csv('/Users/lupitatovar/Cool-Worlds/kois_JR.csv')
jr

,KIC,KOI,Kepler-ID,TTVperiod,TTVperiod_e,Epoch,Epoch_e,RpRs,RpRs_p,RpRs_m,...,[M/H]_e,StatusFlag,rhostar_model_lc,Epoch_lc,Per_lc,b_lc,RpRs_lc,lcflag,ttvper_cor,ttvper_cor_err
0,11446443,1.01,Kepler-1 b,2.470613,2.000000e-08,787.064865,0.000009,0.123869,0.000066,-0.000062,...,0.041,PPPP,1.83550,55.763305,2.470613,0.8178,0.123864,0,0.000000e+00,1.000000e-08
1,10666592,2.01,Kepler-2 b,2.204735,5.000000e-08,788.535431,0.000017,0.075184,0.000008,-0.000009,...,0.042,PPPP,0.40817,54.358544,2.204735,0.0005,0.075181,0,2.000000e-08,2.000000e-08
2,10748390,3.01,Kepler-3 b,4.887803,2.400000e-07,786.095759,0.000082,0.057856,0.000471,-0.000123,...,0.084,PPPP,3.65395,57.813080,4.887803,0.1028,0.057902,0,2.000000e-08,5.000000e-08
3,3861595,4.01,Kepler-1658 b,3.849371,1.160000e-06,787.262929,0.000525,0.039553,0.000820,-0.001229,...,0.082,PPPP,0.21898,90.526690,3.849372,0.9165,0.039657,0,-1.100000e-07,3.000000e-07
4,8554498,5.01,NaN,4.780328,8.700000e-07,787.803519,0.000159,0.036536,0.000218,-0.000187,...,0.158,PPPP,0.34945,65.974079,4.780328,0.9512,0.036532,0,-1.000000e-08,1.800000e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9718,11610352,8420.01,NaN,4.377549,2.119800e-04,790.506658,0.021818,0.007583,0.001149,-0.001149,...,0.140,SNNN,1.08218,68.230876,4.377417,0.0659,0.007212,0,5.790000e-06,4.843000e-05
9719,11653065,8421.01,NaN,0.284970,9.300000e-07,794.013069,0.004723,0.006505,0.000606,-0.000371,...,0.151,PNNN,0.43429,64.766425,0.284974,0.4890,0.006649,0,-1.120000e-05,3.260000e-06
9720,11867853,8422.01,NaN,0.642227,3.070000e-06,794.122100,0.007560,0.007498,0.003033,-0.001300,...,0.141,SNNN,1.09801,65.175282,0.642237,0.3420,0.007224,0,-6.440000e-06,4.780000e-06
9721,12069878,8423.01,NaN,42.038145,2.602350e-03,753.695994,0.011121,0.010799,0.001395,-0.001014,...,0.129,SNNN,23.73683,81.091603,42.037535,0.2816,0.010708,0,7.540000e-06,6.191000e-05


In [5]:
jr.columns

Index(['KIC', 'KOI', 'Kepler-ID', 'TTVperiod', 'TTVperiod_e', 'Epoch',
       'Epoch_e', 'RpRs', 'RpRs_p', 'RpRs_m', 'b', 'b_ep', 'b_em',
       'rhostar_model', 'rhostar_model_ep', 'rhostar_model_em', 'u1', 'u2',
       'nTTobs', 'nTT', 'TTVflag', 'tdepth', 'tdepth_e', 'tdur', 'tdur_e',
       'tduravg', 'tduravg_e', 'radius', 'radius_ep', 'radius_em', 'SNR',
       'SNRwoTTV-SNRwTTV', 'MES', 'chisqwttv', 'chisqwottv', 'dchisq_inj',
       'adrs', 'adrs_ep', 'adrs_em', 'incl', 'incl_ep', 'incl_em', 's0',
       's0_ep', 's0_em', 'kepmag', 'stellar_source', 'rhostar', 'rhostar_ep',
       'rhostar_em', 'teff', 'teff_e', 'rstar', 'rstar_ep', 'rstar_em',
       'mstar', 'mstar_ep', 'mstar_em', 'logg', 'logg_ep', 'logg_em', '[M/H]',
       '[M/H]_e', 'StatusFlag', 'rhostar_model_lc', 'Epoch_lc', 'Per_lc',
       'b_lc', 'RpRs_lc', 'lcflag', 'ttvper_cor', 'ttvper_cor_err'],
      dtype='object')

Making a mass, radius, isolation mask

- Mass <= 8 $M_{Earth}$
- Radius <= 2 $R_{Earth}$
- Insolation <= 1 $I_{Earth}$

In [7]:
jr['StatusFlag'].values

array(['PPPP', 'PPPP', 'PPPP', ..., 'SNNN', 'SNNN', 'SNNN'], dtype=object)

In [8]:
radius_mask = np.where((jr['radius'].values <= 2))[0]
#there is no mass or insolation column...
len(radius_mask)

3866